In [33]:
#Assginment 1: Fast Trajectory Replanning
#CS 440: Intro to Artificial Intelligence
#Due date: 26th Feburary 2018
#authors: Ayush Joshi, Nikhil Mathur, and Dan Snyder
import heapq
import pygame
import random
import os
import math
import time
from random import *
from random import randint
from random import randrange
from ast import literal_eval as make_tuple
from IPython.core.debugger import set_trace

In [34]:
#Convention for the Environment
# Initialize Grid
# 0 is blocked and 1 is unblocked for track_maze
# 0 is unvisited and 1 is visited for track_maze
# gn = get neighbour
# nn = new neighbour
# p = parent

In [35]:
# Initialize Variables

blockwidth = 8  # Drawing dimensions of block
GridCols = 101   # No of columns
GridRows = 101   # No of rows

maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
track_maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
first_parent_x = 0
first_parent_y = 0
last_parent_x = 0
last_parent_y = 0
cursor_x = 0
cursor_y = 0
i = 0

#main tree
tree = {}

In [36]:
#get the string for the cell location
def get_String(x,y):
    return str(x)+","+str(y)

#get the coordinates for the cell.
def get_coord(cell):
    li = cell.split(",")
    return int(li[0]),int(li[1])

#check if the cell is visited or not.
def is_visited(x,y):
    if x is None and y is None:
        return False
    x= int(x)
    y=int(y)
    if track_maze[x][y] == 1:
        return True
    return False

def is_first(x,y):
    if x == first_parent_x and y == first_parent_y:
        return True
    else:
        return False

# get the key for the value from the tree
def get_key(value):
    key_value_dict = {}
    for i in tree.items():
        a = i[0]
        b = i[1]
        if a and b:
            for item in b:
                key_value_dict[str(item)] = str(a)
    return key_value_dict[value]

In [37]:
def valid_coordinates(x,y):
    if x is None and y is None:
        return False
    if 0<= x <GridCols and 0<= y <GridRows:
        return True
    return False
    
#get the neighbours for a cell.
def get_neighbour(x,y,point):
    x= int(x)
    y= int(y)
    point = int(point)
    if point == 0 and valid_coordinates(x-1,y):
        return x-1,y
    if point == 1 and valid_coordinates(x+1,y):
        return x+1,y
    if point == 2 and valid_coordinates(x,y-1):
        return x,y-1
    if point == 3 and valid_coordinates(x,y+1):
        return x,y+1
    return None,None

#check if the cell has neighbours that are not visited.
def validate_neighbour(x,y):
    x=int(x)
    y=int(y)
    tf_value = False    
        
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            tf_value = True
    except:
        pass  
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            tf_value = True
    except:
        pass
        
    return tf_value

#get the neighbour for a cell
def get_validate_neighbour(x,y):
    x = int(x)
    y = int(y)
    
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            return x-1, y
    except:
        pass
    
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            return x+1, y
    except:
        pass
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            return x, y-1
    except:
        pass
    
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            return x, y+1
    except:
        pass
    
    return None, None


In [38]:
#build_tree returns last child which was visited
def build_tree(p_x,p_y):
    nn_x = None
    nn_y = None
    last_p_x = p_x
    last_p_y = p_y

    while validate_neighbour(p_x,p_y) is True:
        ra = randint(0,3)
        nn_x,nn_y = get_neighbour(p_x,p_y,ra)
        if nn_x is None or nn_y is None or is_visited(nn_x,nn_y):
            while validate_neighbour(p_x,p_y):
                ran = randint(0,3)
                nn_x,nn_y = get_neighbour(p_x,p_y,ran)
                if valid_coordinates(nn_x,nn_y):
                    if is_visited(nn_x,nn_y) == False:
                        break

        if valid_coordinates(nn_x,nn_y) and is_visited(nn_x,nn_y) == False:
            tree[get_String(p_x,p_y)] = [get_String(nn_x,nn_y)] 
            track_maze[int(nn_x)][int(nn_y)] = 1
            last_p_x = nn_x
            last_p_y = nn_y
            rand_block_unblock = randint(0,100)
            #if rand_block_unblock < 30:
                #track_maze[nn_x][nn_y] = 1 # to be changed to 0, using to track
            #else:
                #track_maze[nn_y][nn_y] = 1
            p_x,p_y = nn_x,nn_y
    return last_p_x, last_p_y


In [39]:
#back_track returns new child for a parent with unvisited cell
def back_track(last_x,last_y):
    child_x = last_x
    child_y = last_y
    while is_first(child_x,child_y) == False:
        parent = get_key(get_String(child_x,child_y))
        parent_x,parent_y = get_coord(parent)
        if validate_neighbour(parent_x,parent_y):
            nnn_x,nnn_y = get_validate_neighbour(parent_x,parent_y)
            if nnn_x is not None and nnn_y is not None and is_visited(nnn_x,nnn_y) == False:
                try:
                    newlist = []
                    for item in tree[get_String(parent_x,parent_y)]:
                        newlist.append(item)    
                    newlist.append(get_String(int(nnn_x),int(nnn_y)))
                    #update the list
                    tree[get_String(parent_x,parent_y)] = newlist
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
                except:
                    tree[get_String(parent_x,parent_y)] = [get_String(int(nnn_x),int(nnn_y))]
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
                return get_String(int(nnn_x),int(nnn_y))
        child_x,child_y = parent_x,parent_y
    return None

In [40]:
# Draw Screen
def drawScreen(GridSurface):
    GameScreen.blit(GridSurface,(0,0))
    pygame.display.update()

In [41]:
# Draw Grid
def drawGrid(myGridSurface):
    myGridSurface.fill((255,255,255))
    for x in range(len(track_maze)):
        for y in range(len(track_maze[x])):
            if track_maze[x][y] == 0: 
                pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
                pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
            elif track_maze[x][y] == 1: 
                pygame.draw.rect(myGridSurface, (255,255,255), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
                pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
    myGridSurface = myGridSurface.convert()
    return myGridSurface

In [42]:
# Make the grid
#areacoordinates = makeGrid()

In [44]:
## For visual
pygame.init()
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())
myfont = pygame.font.SysFont("monospace", 15)
start_x,start_y,goal_x,goal_y = 0,0,0,0
#generateStartFinish()

In [45]:
# Make Random Grid Visual
def makeGrid_visual():
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        global first_parent_x
        first_parent_x = p_x
        global first_parent_y
        first_parent_y = p_y
        ra = randint(0,3)
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    print("first parent",first_parent_x,first_parent_y)
    print("first child", gn_x,gn_y)
    track_maze[int(p_x)][int(p_y)] = 1
    track_maze[int(gn_x)][int(gn_y)] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    
    while True:
        #add space for text later
        GameScreen = pygame.display.set_mode((GridCols*blockwidth+20,GridRows*blockwidth+20))
        GridSurface = pygame.Surface(GameScreen.get_size())
        GridSurface = drawGrid(GridSurface)
        
        final_path = []
        closed_list = []
        cell_costs = []
        priority_list = []
        heuristic_list = []
        path_cost = 0
        elapsed_time = 0
        drawmode = 0
        nodes_expanded = 0
        #print(GridSurface)
        drawScreen(GridSurface)
        #time.sleep(2)
        #pygame.quit()
        #i = i +1
        #print("Tree",tree)
        #print("\n")
        new_x,new_y = build_tree(p_x,p_y)
        global last_parent_x
        global last_parent_y
        if last_parent_x == new_x and last_parent_y == new_y:
            break
        last_parent_x = new_x
        last_parent_y = new_y
        print("back to main",new_x,new_y)
        if new_x == first_parent_x and new_y == first_parent_y:
            return None
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == first_parent_x and p_y == first_parent_y:
                    break 
            if new_parent is None:
                return None
    return None


In [46]:

# Make Random Grid Visual
def makeGrid():
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        global first_parent_x
        first_parent_x = p_x
        global first_parent_y
        first_parent_y = p_y
        ra = randint(0,3)
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    track_maze[p_x][p_y] = 1
    track_maze[p_x][p_y] = 1
    track_maze[gn_x][gn_y] = 1
    track_maze[gn_x][gn_y] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    
    
    while True:
        i = i +1
        #print("Tree",tree)
        print("\n")
        new_x,new_y = build_tree(p_x,p_y)
        global last_parent_x
        global last_parent_y
        if last_parent_x == new_x and last_parent_y == new_y:
            break
        
        last_parent_x = new_x
        last_parent_y = new_y
        
        print("back to main",new_x,new_y)
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == first_parent_x and p_y == first_parent_y:
                    break 
            if new_parent is None:
                return None
    return None


In [47]:
makeGrid_visual()

first
first parent 34 99
first child 33 99
back to main 39 98
Back track found 40,100
back to main 51 98
Back track found 49,98
back to main 50 100
Back track found 49,99
back to main 49 99
Back track found 47,100
back to main 46 100
Back track found 50,97
back to main 50 96
Back track found 55,100
back to main 62 93
Back track found 63,95
back to main 64 95
Back track found 60,95
back to main 58 95
Back track found 56,95
back to main 58 79
Back track found 57,80
back to main 57 82
Back track found 55,82
back to main 54 83
Back track found 52,83
back to main 48 83
Back track found 49,82
back to main 56 80
Back track found 55,79
back to main 46 75
Back track found 46,77
back to main 45 73
Back track found 46,72
back to main 40 71
Back track found 42,70
back to main 57 74
Back track found 61,77
back to main 61 78
Back track found 61,82
back to main 57 84
Back track found 57,86
back to main 58 89
Back track found 57,90
back to main 57 93
Back track found 56,92
back to main 56 92
Back trac

back to main 50 60
Back track found 42,56
back to main 45 53
Back track found 46,56
back to main 47 58
Back track found 49,55
back to main 49 55
Back track found 46,51
back to main 47 45
Back track found 45,46
back to main 41 44
Back track found 39,44
back to main 39 45
Back track found 37,48
back to main 30 40
Back track found 32,39
back to main 27 40
Back track found 27,42
back to main 19 50
Back track found 20,51
back to main 20 52
Back track found 22,52
back to main 24 53
Back track found 24,50
back to main 24 50
Back track found 27,51
back to main 25 46
Back track found 24,45
back to main 26 43
Back track found 23,43
back to main 22 49
Back track found 20,48
back to main 21 41
Back track found 22,39
back to main 24 38
Back track found 25,36
back to main 24 33
Back track found 26,33
back to main 26 30
Back track found 28,30
back to main 28 31
Back track found 27,34
back to main 26 34
Back track found 26,36
back to main 25 28
Back track found 24,30
back to main 23 30
Back track foun

back to main 15 91
Back track found 15,94
back to main 15 94
Back track found 13,93
back to main 13 93
Back track found 12,91
back to main 11 87
Back track found 13,89
back to main 13 90
Back track found 10,91
back to main 11 92
Back track found 9,91
back to main 9 91
Back track found 19,96
back to main 20 92
Back track found 21,95
back to main 22 95
Back track found 25,93
back to main 28 92
Back track found 27,91
back to main 24 87
Back track found 26,87
back to main 34 89
Back track found 32,88
back to main 32 86
Back track found 31,87
back to main 27 87
Back track found 34,85
back to main 33 83
Back track found 35,81
back to main 35 81
Back track found 31,84
back to main 31 84
Back track found 30,85
back to main 29 87
Back track found 28,84
back to main 27 84
Back track found 25,91
back to main 26 91
Back track found 25,92
back to main 25 92
Back track found 29,94
back to main 26 94
Back track found 26,80
back to main 26 80
Back track found 24,79
back to main 24 78
Back track found 

back to main 99 19
Back track found 93,28
back to main 92 27
Back track found 96,21
back to main 98 22
Back track found 95,16
back to main 95 15
Back track found 98,6
back to main 98 6
Back track found 100,4
back to main 100 0
Back track found 98,0
back to main 97 0
Back track found 95,0
back to main 94 0
Back track found 96,3
back to main 95 4
Back track found 96,5
back to main 96 5
Back track found 96,2
back to main 96 2
Back track found 100,3
back to main 98 3
Back track found 95,8
back to main 93 9
Back track found 92,8
back to main 92 8
Back track found 95,10
back to main 95 10
Back track found 92,12
back to main 91 14
Back track found 92,13
back to main 92 13
Back track found 86,13
back to main 86 13
Back track found 82,14
back to main 82 14
Back track found 83,12
back to main 82 12
Back track found 80,13
back to main 82 7
Back track found 83,8
back to main 83 8
Back track found 80,7
back to main 80 7
Back track found 81,5
back to main 81 4
Back track found 83,4
back to main 83 4

Back track found 21,33
back to main 18 34
Back track found 22,36
back to main 22 35
Back track found 21,45
back to main 21 46
Back track found 18,46
back to main 15 46
Back track found 18,45
back to main 18 44
Back track found 18,48
back to main 19 48
Back track found 25,48
back to main 26 49
Back track found 28,51
back to main 29 51
Back track found 22,54
back to main 22 54
Back track found 28,53
back to main 28 53
Back track found 30,48
back to main 29 49
Back track found 28,45
back to main 28 42
Back track found 27,45
back to main 27 45
Back track found 29,45
back to main 29 45
Back track found 25,41
back to main 24 41
Back track found 28,39
back to main 28 38
Back track found 33,39
back to main 34 39
Back track found 33,42
back to main 35 41
Back track found 37,41
back to main 38 39
Back track found 37,44
back to main 37 44
Back track found 34,46
back to main 34 44
Back track found 33,46
back to main 33 46
Back track found 45,48
back to main 45 51
Back track found 49,51
back to mai

In [48]:
pygame.quit()

In [ ]:
#print(get_key("46,2"))
print(first_parent_x, first_parent_y)
#back_track(46,2)
print(back_track(2,2))


In [ ]:
track_maze

In [ ]:
# Main loop
running = True
pygame.init()
myfont = pygame.font.SysFont("monospace", 15)
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())

time.sleep(2)
GridSurface = drawGrid(GridSurface)
start_x,start_y,goal_x,goal_y = generateStartFinish()
final_path = []
closed_list = []
cell_costs = []
priority_list = []
heuristic_list = []
path_cost = 0
elapsed_time = 0
drawmode = 0
nodes_expanded = 0

MySearch = AStarSearch() # Initialize Object

while(running):
	# Get Input
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			running = False
		elif event.type == pygame.KEYDOWN:
			if event.key == pygame.K_ESCAPE:
				running = False
			elif event.key == pygame.K_g:
				track_maze = [['1' for y in range(GridRows)] for x in range(GridCols)]
				areacoordinates = makeGrid()
				GridSurface = drawGrid(GridSurface)
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new map")
			elif event.key == pygame.K_e:
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new start and finish points")
			elif event.key == pygame.K_s:
				# Save map: get filename
				filename = raw_input("Save map to: ")
				with open(os.path.join("./gen",filename),"w") as mapfile:
					mapfile.write(str((start_x,start_y)))		# Write start
					mapfile.write("\n")
					mapfile.write(str((goal_x,goal_y)))			# Write goal
					mapfile.write("\n")
					
					for area in areacoordinates:				# Write hard to traverse area centers
						mapfile.write(str((area[0],area[1])))		
						mapfile.write("\n")
					
					for y in range(len(track_maze[x])):				# Write each cell
						for x in range(len(track_maze)):					
							mapfile.write(str(track_maze[x][y]))		
						mapfile.write("\n")
					
					mapfile.close()
				print("Map saved!")
			elif event.key == pygame.K_l:
				# Load map: get filename
				filename = raw_input("Load map from: ")
				with open(os.path.join("./gen",filename),"r") as mapfile: #changed to allow using /maps folder
					new_start = make_tuple(mapfile.readline())
					start_x = new_start[0]
					start_y = new_start[1]
					new_goal = make_tuple(mapfile.readline())
					goal_x = new_goal[0]
					goal_y = new_goal[1]
					
					areacoordinates = []
					
					for i in range(8):
						new_area = make_tuple(mapfile.readline())
						areacoordinates.append((new_area[0],new_area[1]))
					
					for y in range(len(track_maze[x])):				# Read each cell
						for x in range(len(track_maze)):					
							track_maze[x][y] = mapfile.read(1)
						mapfile.read(1)
					
					mapfile.close()
				final_path = []
				closed_list = []
				cell_costs = []
				priority_list = []
				heuristic_list = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				GridSurface = drawGrid(GridSurface)
				print("Map loaded!")
			elif event.key == pygame.K_UP:
				if cursor_y-1 >= 0:
					cursor_y -= 1
			elif event.key == pygame.K_LEFT:
				if cursor_x-1 >= 0:
					cursor_x -= 1
			elif event.key == pygame.K_RIGHT:
				if cursor_x+1 < GridCols:
					cursor_x += 1
			elif event.key == pygame.K_DOWN:
				if cursor_y+1 < GridRows:
					cursor_y += 1
			elif event.key == pygame.K_v:
				# draw closed list
				if drawmode == 0:
					drawmode = 1
				else:
					drawmode = 0
			elif event.key == pygame.K_a:		# -------- A* Search --------
				choice = -1
				while int(choice) < 1 or int(choice) > 6:
					choice = raw_input ("Enter (1) for Manhattan distance, (2) for Euclidean distance, (3) for Octile distance, (4) for Chebyshev distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				MySearch = AStarSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,choice)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_u:		# -------- Uniform Cost Search --------
				MySearch = UniformCostSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,1)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_w:		# -------- Weighted A* Search --------
				choice = -1 #heuristic choice
				weight = 0 #weight of heuristic

				while (int(choice) < 1 or int(choice) > 6):
					choice = raw_input("Enter (1) for Manhattan distance, (2) for Euclidean Distance, (3) for Octile Distance, (4) for Chebyshev Distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				while float(weight) < 1:
					weight = raw_input("Enter the selected weight for Weighted A* - must be >= 1: ")
				
				MySearch = WeightedAStarSearch(weight)
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y, choice)
				elapsed_time = time.time() - start_time
				
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_q:		# -------- Sequential A* Search --------
				MySearch = AStarSearch()

				
		#print(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)
		drawScreen(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)

pygame.quit()